In [1]:
import taichi as ti

# ti.init(debug=True)
ti.init(arch=ti.gpu)

max_num_particles = 256

deltatime = 1e-3

num_particles = ti.var(ti.i32, shape=())
spring_stiffness = ti.var(ti.f32, shape=())
paused = ti.var(ti.i32, shape=())
damping = ti.var(ti.f32, shape=())

particle_mass = 1
bottom_y = 0.05
top_y = 0.95
left_x = 0.05
right_x = 0.95

pos = ti.Vector(2, dt=ti.f32, shape=max_num_particles)
vol = ti.Vector(2, dt=ti.f32, shape=max_num_particles)

A = ti.Matrix(2, 2, dt=ti.f32, shape=(max_num_particles, max_num_particles))
b = ti.Vector(2, dt=ti.f32, shape=max_num_particles)
r = ti.Vector(2, dt=ti.f32, shape=())
dv = ti.Vector(2,dt=ti.f32, shape=max_num_particles)
new_dv = ti.Vector(2,dt=ti.f32, shape=max_num_particles)

connection_radius = 0.15
rest_length = ti.var(ti.f32, shape=(max_num_particles, max_num_particles))
paused[None] = 0

@ti.kernel
def new_particle(pos_x: ti.f32, pos_y: ti.f32):
    id = num_particles[None]
    num_particles[None] += 1
    pos[id] = [pos_x, pos_y]
    vol[id] = [0.0, 0.0]
    
    for i in range(num_particles-1):
        dist = (pos[id] - pos[i]).norm()
        if dist < connection_radius:
            rest_length[id,i] = 0.1
            rest_length[i,id] = 0.1
            
@ti.kernel
def substep():
    n = num_particles[None]
    for i in range(n):
        vol[i] *= ti.exp(-deltatime * damping[None])
        total_force = gravity * particle_mass
        for j in range(n):
            if rest_length[i,j] != 0:
                pos_ij = pos[i] - pos[j]
                total_force += -spring_stiffness[None] * (pos_ij.norm() - rest_length[i,j]) * pos_ij.normalized()
        vol[i] += deltatime * total_force / particle_mass
        
    for i in range(n):
        if pos[i].y < bottom_y:
            pos[i].y = bottom_y
            vol[i].y = 0.0
            
    for i in range(n):
        pos[i] += deltatime * vol[i]
                
            
gui = ti.GUI('Mass Spring System', res=(512, 512), background_color=0xdddddd) #

spring_stiffness[None] = 10000
damping[None] = 20
gravity = ti.Vector([0.0,-9.8])

new_particle(0.3, 0.3)
new_particle(0.3, 0.4)
new_particle(0.4, 0.4)
while True:
    for e in gui.get_events(ti.GUI.PRESS):
        if e.key in [ti.GUI.ESCAPE, ti.GUI.EXIT]:
            exit()
        elif e.key == ti.GUI.LMB:
            new_particle(e.pos[0], e.pos[1])
        elif e.key == 'c':
            num_particles[None] = 0
            rest_length.fill(0)
        elif e.key == 'p':
            if paused[None] == 0:
                paused[None] = 1
            elif paused[None] == 1:
                paused[None] =0
            
    if paused[None] == 0:
        for step in range(10):
            substep()
    
    Pos = pos.to_numpy()
    gui.circles(Pos[:num_particles[None]], color=0xffaa77, radius=5)
    gui.line(begin=(0.0, bottom_y), end=(1.0, bottom_y), color=0x0, radius=1)
    for i in range(num_particles[None]):
        for j in range(i + 1, num_particles[None]):
            if rest_length[i, j] != 0:
                gui.line(begin=Pos[i], end=Pos[j], radius=2, color=0x445566)
    gui.text(content=f'C: clear all; Space: pause', pos=(0, 0.95), color=0x0)
    gui.text(content=f'S: Spring stiffness {spring_stiffness[None]:.1f}', pos=(0, 0.9), color=0x0)
    gui.text(content=f'D: damping {damping[None]:.2f}', pos=(0, 0.85), color=0x0)
    gui.show()
    

[Taichi] mode=release
[Taichi] version 0.6.22, llvm 10.0.0, commit f5283775, python 3.7.4
[Taichi] Interactive shell detected: IPython ZMQInteractiveShell
[Taichi] Starting on arch=cuda


RuntimeError: Window close button clicked, exiting... (use `while gui.running` to exit gracefully)